In [1]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from scipy.linalg import eigh
import qutip as qt
import sympy as sym

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
reload(sq_ext)
reload(sq)
reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

# Comparison of sqcircuit, numeric and spin-boson models vs truncation

In [2]:
Cc = 0.1
n_eig = 3
# nmax_r_values = np.array([7, 7, 9,  9, 11, 11, 13, 13, 15])
# nmax_f_values = np.array([7, 9, 9, 11, 11, 13, 13, 15, 15])
nmax_r_values = np.array([3, 3, 3, 3, 3])
nmax_f_values = np.array([5, 6, 7, 8, 9])

E_sq = []
E_n  = []

for nmax_r, nmax_f in zip(nmax_r_values, nmax_f_values):
    print(nmax_f)
    
    H_n = sq_ext.hamiltonian_qubit_C_qubit_C_qubit(nmax_r=nmax_r, nmax_f=nmax_f, Cc=Cc, periodic=False)
    E = sq_ext.diag(H_n, n_eig, out='GHz')[0]
    E_n.append(E-E[0])
    del(H_n)
    
    if nmax_f >= 9:
        continue
    else:
        # H_sq = sq_ext.sq_qubit_C_qubit_C_qubit(Cc, nmax_r=nmax_r, nmax_f=nmax_f)
        H_sq = sq_ext.sq_qubit_C_qubit_C_qubit_not_periodic(Cc, nmax_r=nmax_r, nmax_f=nmax_f)
        H_sq.diag(n_eig)
        E_sq.append(H_sq.efreqs-H_sq.efreqs[0])
        del(H_sq)
    
E_sq = np.array(E_sq )
E_n  = np.array(E_n  )

In [ ]:
fig, [ax1,ax2] = plt.subplots(ncols=2)

for i, ax in enumerate([ax1,ax2]):
    ax.plot(E_sq[:,i+1], 'r')
    ax.plot(E_n [:,i+1], 'g')

fig.show()

In [ ]:
Cc_values = np.geomspace( 1e-3, 1, 10)
n_eig = 3
nmax_r = 4
nmax_f = 12 

E_sq = []
E_n  = []
E_sb = []

for Cc in Cc_values:
    print(Cc)
    
    H_n = sq_ext.hamiltonian_qubit_C_qubit(nmax_r=nmax_r, nmax_f=nmax_f, Cc=Cc)
    E = sq_ext.diag(H_n, n_eig, out='GHz')[0]
    E_n.append(E-E[0])
    del(H_n)

    H_sb = sq_ext.spin_boson_qubit_C_qubit(nmax_r=nmax_r, nmax_f=nmax_f, Cc=Cc)
    E = eigh(H_sb, eigvals_only=True)
    E_sb.append(E-E[0])
    del(H_sb)
    
    H_sq = sq_ext.sq_qubit_C_qubit(Cc, nmax_r=nmax_r, nmax_f=nmax_f)
    H_sq.diag(n_eig)
    E_sq.append(H_sq.efreqs-H_sq.efreqs[0])
    del(H_sq)
    
E_sq = np.array(E_sq )
E_n  = np.array(E_n  )
E_sb = np.array(E_sb )   

In [ ]:
fig, [ax1, ax2] = plt.subplots(ncols=2, dpi =200)

for i, ax in enumerate([ax1, ax2]):
    ax.plot(Cc_values, E_sq[:, i + 1], 'r')
    ax.plot(Cc_values, E_n[:, i + 1], '--g')
    ax.plot(Cc_values, E_sb[:, i + 1], 'b')
    
ax.set_xscale('log')
fig.tight_layout()
fig.show()

In [31]:
sq_ext = reload(sq_ext)
Cc = 1e-1
nmax_r = 4
nmax_f = 12 
H_0 = sq_ext.hamiltonian_qubit_C_qubit_C_qubit(Cc=0 , nmax_r=nmax_r, nmax_f=nmax_f, only_outer=True)
H = sq_ext.hamiltonian_qubit_C_qubit_C_qubit(Cc=0 , nmax_r=nmax_r, nmax_f=nmax_f, only_outer=True)
# H   = sq_ext.hamiltonian_qubit_C_qubit_C_qubit(Cc=Cc, nmax_r=nmax_r, nmax_f=nmax_f)

In [38]:
ψ_0 = qt.Qobj(H).eigenstates(sparse=True, eigvals=4, phase_fix=0)[1]

In [39]:
ψ_0

In [36]:
n_eig = 4

H_eff = np.zeros((n_eig, n_eig), dtype=complex)

for i in range(n_eig):
    for j in range(n_eig):
        H_eff[i, j] = (ψ_0[i].dag()
                       * H * ψ_0 [j]).data[0, 0]

In [37]:
H_eff/2/np.pi/1e9

In [24]:
ψ_0.dag() * qt.Qobj(H) * ψ_0

In [3]:
H_eff = sq_ext.H_eff_p1a(H_0, H, 4)

In [18]:
np.round(H_eff,3)

In [22]:
sq_ext.diag(H_0, 2, out='GHz')[0]

In [23]:
sq_ext.diag(H, 2, out='GHz')[0]

In [36]:
sq_ext.diag(H, 2, out='GHz')[0]

In [32]:
H_eff = sq_ext.H_eff_SWT(H_0, H, 4)

In [34]:
sq_ext.diag(H_eff, 2, out='GHz')[0]